#**Covid-19 Detection Using CNN**

###**Installing & Importing Libraries**

In [ ]:
%%shell
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import opendatasets as od
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import shutil
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MaxPooling2D, Flatten, Conv2D, Dense, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib inline

###**Preparing the files**

In [ ]:
# {"username":"ashleyjejigeorge","key":"2f407cf64a34212d7a8d87a0b4d8d7fd"}
od.download('''https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database''')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ashleyjejigeorge
Your Kaggle Key: ··········


100%|██████████| 778M/778M [00:11<00:00, 69.4MB/s]


In [ ]:
dir_path = '/content/covid19-radiography-database/COVID-19_Radiography_Dataset'
os.chdir(dir_path)
os.listdir()

['Viral Pneumonia',
 'Normal',
 'Normal.metadata.xlsx',
 'Viral Pneumonia.metadata.xlsx',
 'Lung_Opacity.metadata.xlsx',
 'README.md.txt',
 'COVID',
 'Lung_Opacity',
 'COVID.metadata.xlsx']

In [ ]:
# Deleting files that are not required
!mv 'Viral Pneumonia' Viral_Pneumonia
!mv 'Viral Pneumonia.metadata.xlsx' Viral_Pneumonia.metadata.xlsx
!rm README.md.txt
!rm Viral_Pneumonia.metadata.xlsx
!rm Lung_Opacity.metadata.xlsx
shutil.rmtree('Lung_Opacity')
shutil.rmtree('Viral_Pneumonia')

In [ ]:
os.listdir()

['Normal', 'Normal.metadata.xlsx', 'COVID', 'COVID.metadata.xlsx']

In [ ]:
covid = pd.read_excel('COVID.metadata.xlsx')
covid.head()

,FILE NAME,FORMAT,SIZE,URL
0,COVID-1,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
1,COVID-2,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
2,COVID-3,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
3,COVID-4,PNG,256*256,https://sirm.org/category/senza-categoria/covi...
4,COVID-5,PNG,256*256,https://sirm.org/category/senza-categoria/covi...


In [ ]:
normal = pd.read_excel('Normal.metadata.xlsx')
normal.head()

,FILE NAME,FORMAT,SIZE,URL
0,NORMAL-1,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
1,NORMAL-2,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
2,NORMAL-3,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
3,NORMAL-4,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...
4,NORMAL-5,PNG,256*256,https://www.kaggle.com/c/rsna-pneumonia-detect...


In [ ]:
covid['label'] = 1
normal['label'] = 0
covid = covid[['FILE NAME', 'label']]
normal = normal[['FILE NAME', 'label']]
SAMPLE_SIZE = 3616
df_0 = covid.sample(SAMPLE_SIZE, random_state=26)
df_1 = normal.sample(SAMPLE_SIZE, random_state=26)

# concat dataframes
data = pd.concat([df_0, df_1], axis=0).reset_index(drop=True)

# check numbers of each label
data['label'].value_counts()

1    3616
0    3616
Name: label, dtype: int64

In [ ]:
data = shuffle(data)
data.head()

,FILE NAME,label
4853,NORMAL-6646,0
1469,COVID-1922,1
4931,NORMAL-274,0
3336,COVID-2796,1
4011,NORMAL-6619,0


In [ ]:
df_train, df_test = train_test_split(data, test_size=0.20, random_state=26)

print(df_train.shape)
print(df_test.shape)

(5785, 2)
(1447, 2)


In [ ]:
df_train['label'].value_counts()

1    2895
0    2890
Name: label, dtype: int64

In [ ]:
df_test['label'].value_counts()

0    726
1    721
Name: label, dtype: int64

In [ ]:
os.mkdir('train')
os.mkdir('train/covid')
os.mkdir('train/normal')
os.mkdir('test')
os.mkdir('test/covid')
os.mkdir('test/normal')
os.mkdir('test_data')
os.mkdir('train_data')
os.listdir()

['Normal',
 'train',
 'Normal.metadata.xlsx',
 'COVID',
 'test',
 'COVID.metadata.xlsx',
 'test_data',
 'train_data']

In [ ]:
# Expriment

train_list = list(df_train['FILE NAME'])
test_list = list(df_test['FILE NAME'])
train_dir = 'train_data'
test_dir = 'test_data'

# Copy images to train_dir folder
for image in train_list:
    
    # add .png extension 
    filename = image + '.png'
    # get the label for a certain image
    target = int(data.loc[data['FILE NAME'] == image, ['label']].values)
    
    # match the target with the folder's name and source path of the image
    if target == 1:
        label = 'covid'
        src = os.path.join('COVID/images', filename)
        
    if target == 0:
        label = 'normal'
        # As 'FILE NAME's in .xlsx file begins with 'NORMAL' but real file names begin with 'Normal'
        filename = filename.capitalize()
        src = os.path.join('Normal/images', filename)

    # destination path to image
    dst = os.path.join(train_dir, filename)
    
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)

# Copy images to test_dir folder
for image in test_list:
    
    # add .png extension 
    filename = image + '.png'
    # get the label for a certain image
    target = int(data.loc[data['FILE NAME'] == image, ['label']].values)
    
    # match the target with the folder's name and source path of the image
    if target == 1:
        label = 'covid'
        src = os.path.join('COVID/images', filename)
        
    if target == 0:
        label = 'normal'
        # As 'FILE NAME's in .xlsx file begins with 'NORMAL' but real file names begin with 'Normal'
        filename = filename.capitalize()
        src = os.path.join('Normal/images', filename)

    # destination path to image
    dst = os.path.join(test_dir, filename)
    
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)

In [ ]:
train_list = list(df_train['FILE NAME'])
test_list = list(df_test['FILE NAME'])
train_dir = 'train'
test_dir = 'test'

# Copy images to train_dir folder
for image in train_list:
    
    # add .png extension 
    filename = image + '.png'
    # get the label for a certain image
    target = int(data.loc[data['FILE NAME'] == image, ['label']].values)
    
    # match the target with the folder's name and source path of the image
    if target == 1:
        label = 'covid'
        src = os.path.join('COVID/images', filename)
        
    if target == 0:
        label = 'normal'
        # As 'FILE NAME's in .xlsx file begins with 'NORMAL' but real file names begin with 'Normal'
        filename = filename.capitalize()
        src = os.path.join('Normal/images', filename)

    # destination path to image
    dst = os.path.join(train_dir, label, filename)
    
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)

# Copy images to test_dir folder
for image in test_list:
    
    # add .png extension 
    filename = image + '.png'
    # get the label for a certain image
    target = int(data.loc[data['FILE NAME'] == image, ['label']].values)
    
    # match the target with the folder's name and source path of the image
    if target == 1:
        label = 'covid'
        src = os.path.join('COVID/images', filename)
        
    if target == 0:
        label = 'normal'
        filename = filename.capitalize()
        src = os.path.join('Normal/images', filename)
        
    # destination path to image
    dst = os.path.join(test_dir, label, filename)
    
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)

In [ ]:
print(len(os.listdir('train/covid')))
print(len(os.listdir('train/normal')))

2895
2890


In [ ]:
print(len(os.listdir('test/covid')))
print(len(os.listdir('test/normal')))

721
726
